In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
df = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\train.parquet')
oos = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oos.parquet')
oot = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oot.parquet')

In [20]:
df.columns

Index(['C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA', 'NHANVIENBIDV',
       'BASE_AUM', 'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX',
       'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M',
       'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'UTILIZATION_RATE', 'CNT_CREDIT_CARDS', 'AMT_CASH_ADVANCE_12M',
       'FLAG_CASH_ADVANCE', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M',
       'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'PTI_RATIO', 'MOB',
       'CNT_OTHER_PRODUCTS', 'LIMIT_TO_INCOME', 'AMT_VAR_6M',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M',
       'OCCUPATION_TYPE', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M',
       'N_AVG_OVERDUE_CBAL_12M', 'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M',
       'BAD

data type 

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137807 entries, 0 to 1137806
Data columns (total 55 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   C_GIOITINH              1137807 non-null  object 
 1   TRINHDO                 1103703 non-null  float64
 2   TTHONNHAN               1137807 non-null  object 
 3   SOHUUNHA                1137807 non-null  int64  
 4   NHANVIENBIDV            1137807 non-null  int64  
 5   BASE_AUM                1137807 non-null  int64  
 6   TUOI                    1137807 non-null  int64  
 7   INCOME                  1080709 non-null  float64
 8   CBAL                    1137807 non-null  int64  
 9   CBALORG                 1137807 non-null  int64  
 10  AFLIMT_MAX              1137807 non-null  int64  
 11  AFLIMT_MIN              1137807 non-null  int64  
 12  AFLIMT_AVG              1137807 non-null  int64  
 13  CBAL_AVG                1137807 non-null  int64  
 14  CB

In [22]:
# Danh sách các biến cần ép kiểu về Category (Nominal & Binary)
categorical_cols = [
    'OCCUPATION_TYPE', 
    'PURCOD_MAX', 
    'PURCOD_MIN',
    'SOHUUNHA', 
    'NHANVIENBIDV',
    'FLAG_SALARY_ACC', 
    'FLAG_DEPOSIT', 
    'FLAG_CASH_ADVANCE',
    'HAS_SHORTTERM_LOAN', # Nếu có trong df final
    'HAS_LONGTERM_LOAN',  # Nếu có trong df final
    'BAD_CURRENT',        # Nếu giữ lại làm feature (thường là drop vì data leakage)
    'XULYNO'              # Nếu giữ lại
]

# Lọc những cột thực sự tồn tại trong df (đề phòng bạn đã drop bớt)
existing_cat_cols = [col for col in categorical_cols if col in df.columns]

# Chuyển đổi
for col in existing_cat_cols:
    df[col] = df[col].astype('str') # Chuyển về string để EBM hiểu là category
    oos[col] = oos[col].astype('str')
    oot[col] = oot[col].astype('str')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137807 entries, 0 to 1137806
Data columns (total 55 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   C_GIOITINH              1137807 non-null  object 
 1   TRINHDO                 1103703 non-null  float64
 2   TTHONNHAN               1137807 non-null  object 
 3   SOHUUNHA                1137807 non-null  object 
 4   NHANVIENBIDV            1137807 non-null  object 
 5   BASE_AUM                1137807 non-null  int64  
 6   TUOI                    1137807 non-null  int64  
 7   INCOME                  1080709 non-null  float64
 8   CBAL                    1137807 non-null  int64  
 9   CBALORG                 1137807 non-null  int64  
 10  AFLIMT_MAX              1137807 non-null  int64  
 11  AFLIMT_MIN              1137807 non-null  int64  
 12  AFLIMT_AVG              1137807 non-null  int64  
 13  CBAL_AVG                1137807 non-null  int64  
 14  CB

target analysis: vẽ biểu đồ phân phối target, xác định imbalance 

In [24]:
data_dict = {
    'TRAIN': df,
    'OOS': oos,
    'OOT': oot
}

results = []

# 3. Tính toán chỉ số cho từng tập
for name, sub_df in data_dict.items():
    # Kiểm tra xem cột target có tồn tại không
    if 'BAD_NEXT_12M' not in sub_df.columns:
        print(f"Cảnh báo: Không tìm thấy cột BAD_NEXT_12M trong tập {name}")
        continue
        
    total_obs = len(sub_df)
    total_bad = sub_df['BAD_NEXT_12M'].sum()
    bad_rate = sub_df['BAD_NEXT_12M'].mean()
    
    results.append({
        'SAMPLE_TYPE': name,
        'Total Obs': total_obs,
        'Total Bad': total_bad,
        'Bad Rate': bad_rate,
        'Bad Rate %': round(bad_rate * 100, 2)
    })

# 4. Tạo bảng tổng hợp (DataFrame Summary)
summary = pd.DataFrame(results).set_index('SAMPLE_TYPE')

# 5. Tính thêm dòng Tổng (Global) để có cái nhìn toàn cảnh
total_row = pd.DataFrame({
    'Total Obs': [summary['Total Obs'].sum()],
    'Total Bad': [summary['Total Bad'].sum()],
    'Bad Rate': [summary['Total Bad'].sum() / summary['Total Obs'].sum()],
    'Bad Rate %': [round((summary['Total Bad'].sum() / summary['Total Obs'].sum()) * 100, 2)]
}, index=['GLOBAL'])

# Gộp dòng tổng vào bảng
summary_final = pd.concat([summary, total_row])

# 6. In kết quả đẹp
print("\n=== KIỂM TRA TỶ LỆ BAD RATE THEO TỪNG TẬP (SPLIT FILES) ===")
print(summary_final[['Total Obs', 'Total Bad', 'Bad Rate %']])

# Check nhanh cảnh báo Imbalance
train_rate = summary.loc['TRAIN', 'Bad Rate %']
if train_rate < 1.0:
    print(f"\n[CẢNH BÁO] Bad Rate tập Train quá thấp ({train_rate}%). Cần cân nhắc Oversampling hoặc chỉnh class_weight.")
elif train_rate > 20.0:
    print(f"\n[CẢNH BÁO] Bad Rate tập Train quá cao ({train_rate}%). Kiểm tra lại định nghĩa Bad.")
else:
    print(f"\n[OK] Tỷ lệ Bad Rate tập Train ổn định ({train_rate}%).")


=== KIỂM TRA TỶ LỆ BAD RATE THEO TỪNG TẬP (SPLIT FILES) ===
        Total Obs  Total Bad  Bad Rate %
TRAIN     1137807      84797        7.45
OOS        300317      18884        6.29
OOT        302113      18147        6.01
GLOBAL    1740237     121828        7.00

[OK] Tỷ lệ Bad Rate tập Train ổn định (7.45%).


data leakage: bỏ các biến tương quan cao với target 

các obs đã xấu sẵn

missing value 

In [25]:
# 1. Tính toán số lượng và tỷ lệ missing
missing_info = df.isnull().sum().reset_index()
missing_info.columns = ['Column', 'Missing Count']
missing_info['Missing %'] = (missing_info['Missing Count'] / len(df)) * 100

# 2. Lọc ra những biến CÓ missing và sắp xếp giảm dần
missing_report = missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing %', ascending=False)

# 3. Hiển thị kết quả
print(f"Tổng số dòng dữ liệu: {len(df):,}")
print("-" * 50)
if missing_report.empty:
    print("Tuyệt vời! Dữ liệu không có missing value nào.")
else:
    print(missing_report.to_string(index=False))

Tổng số dòng dữ liệu: 1,137,807
--------------------------------------------------
          Column  Missing Count  Missing %
COLLATERAL_VALUE         220419  19.372266
          INCOME          57098   5.018250
         TRINHDO          34104   2.997345


ở oos

ở oot 

multicolinearity 

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

X = df.drop('BAD_NEXT_12M', axis=1)
X_oos = oos.drop('BAD_NEXT_12M', axis=1)
X_oot = oot.drop('BAD_NEXT_12M', axis=1)
# 1. Chỉ chọn các biến số (Numeric) để tính toán
numeric_cols = X.select_dtypes(include=[np.number]).columns
corr_matrix = X[numeric_cols].corr().abs() # Lấy trị tuyệt đối (âm hay dương đều là tương quan mạnh)

# 2. Chọn ngưỡng cắt (Threshold)
# Với EBM, ngưỡng an toàn là 0.7. Ngưỡng 0.8 là bắt buộc phải xử lý.
threshold = 0.7

# 3. Lọc ra các cặp biến vượt ngưỡng
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Tạo danh sách chi tiết để bạn dễ quyết định giữ ai/bỏ ai
high_corr_pairs = []
for col in upper.columns:
    high_cols = upper.index[upper[col] > threshold].tolist()
    for row in high_cols:
        val = upper.loc[row, col]
        high_corr_pairs.append((row, col, val))

# Sắp xếp theo độ tương quan giảm dần
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 4. In kết quả
print(f"Tìm thấy {len(high_corr_pairs)} cặp biến tương quan cao (> {threshold}):")
print("-" * 60)
print(f"{'Biến 1 (Giữ?)':<30} | {'Biến 2 (Bỏ?)':<30} | {'Corr':<10}")
print("-" * 60)
for pair in high_corr_pairs:
    print(f"{pair[0]:<30} | {pair[1]:<30} | {pair[2]:.4f}")

# (Tùy chọn) Vẽ Heatmap nếu muốn nhìn tổng quan
# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, cmap='coolwarm')
# plt.show()

Tìm thấy 71 cặp biến tương quan cao (> 0.7):
------------------------------------------------------------
Biến 1 (Giữ?)                  | Biến 2 (Bỏ?)                   | Corr      
------------------------------------------------------------
N_AVG_DEPOSIT_12M              | N_AVG_DD_12M                   | 1.0000
N_AVG_DEPOSIT_6M               | N_AVG_DD_12M                   | 1.0000
N_AVG_DEPOSIT_12M              | N_AVG_DEPOSIT_6M               | 1.0000
MAX_DPD_12M                    | MAX_DPD_12M_OBS                | 1.0000
N_AVG_DD_12M                   | N_AVG_CD_12M                   | 1.0000
CBAL_AVG                       | CBAL_MIN                       | 1.0000
AFLIMT_MIN                     | AFLIMT_AVG                     | 1.0000
N_AVG_DEPOSIT_12M              | N_AVG_CD_12M                   | 1.0000
N_AVG_DEPOSIT_6M               | N_AVG_CD_12M                   | 1.0000
DTI_RATIO                      | CBAL_TO_INC_12MON              | 1.0000
AFLIMT_MAX                

In [27]:
# Danh sách các biến cần loại bỏ để khử đa cộng tuyến
cols_to_drop_final = [
    # 1. Deposit Group
    'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M', 'N_AVG_CD_12M',
    
    # 2. Balance & Limit Group
    'AFLIMT_MIN', 'AFLIMT_AVG', 
    'CBAL', 'CBAL_MAX', 'CBAL_MIN', 'CBALORG',
    
    # 3. Delinquency Group
    'MAX_DPD_12M', 'SUM_ALL_OD_12M', 'N_AVG_OVERDUE_CBAL_12M',
    
    # 4. Ratios Group
    'CBAL_TO_INC_12MON', 'PTI_RATIO', 'COLLATERAL_VALUE',
    
    # 5. Others
    'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX'
]

# Lọc chỉ những cột có trong X để tránh lỗi
cols_to_drop_final = [c for c in cols_to_drop_final if c in X.columns]

# Thực hiện Drop
X = X.drop(columns=cols_to_drop_final)
X_oos = X_oos.drop(columns=cols_to_drop_final)
X_oot = X_oot.drop(columns=cols_to_drop_final)

print(f"Đã bỏ {len(cols_to_drop_final)} biến.")
print(f"Số lượng biến còn lại: {X.shape[1]}")
print("Danh sách biến còn lại:", X.columns.tolist())

Đã bỏ 17 biến.
Số lượng biến còn lại: 37
Danh sách biến còn lại: ['C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA', 'NHANVIENBIDV', 'BASE_AUM', 'TUOI', 'INCOME', 'AFLIMT_MAX', 'CBAL_AVG', 'LTV', 'N_AVG_DEPOSIT_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT', 'UTILIZATION_RATE', 'CNT_CREDIT_CARDS', 'AMT_CASH_ADVANCE_12M', 'FLAG_CASH_ADVANCE', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M', 'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'MOB', 'CNT_OTHER_PRODUCTS', 'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M', 'OCCUPATION_TYPE', 'DURATION_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'REAL_GDP_GROWTH_12M']


In [28]:
drop_oos = [c for c in cols_to_drop_final if c in oos.columns]
drop_oot = [c for c in cols_to_drop_final if c in oot.columns]

# 2. Thực hiện bỏ cột
if drop_oos:
    oos = oos.drop(columns=drop_oos)
    print(f"   - Đã bỏ {len(drop_oos)} biến trong tập OOS.")

if drop_oot:
    oot = oot.drop(columns=drop_oot)
    print(f"   - Đã bỏ {len(drop_oot)} biến trong tập OOT.")

# 3. Kiểm tra lại số lượng cột
print(f"\nKiểm tra số lượng cột sau khi đồng bộ:")
print(f"   - Train (X): {X.shape[1]} biến")
print(f"   - OOS:       {oos.shape[1]} biến (bao gồm cả Target)")
print(f"   - OOT:       {oot.shape[1]} biến (bao gồm cả Target)")

   - Đã bỏ 17 biến trong tập OOS.
   - Đã bỏ 17 biến trong tập OOT.

Kiểm tra số lượng cột sau khi đồng bộ:
   - Train (X): 37 biến
   - OOS:       38 biến (bao gồm cả Target)
   - OOT:       38 biến (bao gồm cả Target)


outlier

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Chọn các biến số (Numeric) để kiểm tra
# Loại bỏ các biến đã chuyển thành object/category ở bước trước
numeric_cols = X.select_dtypes(include=[np.number]).columns

outlier_stats = []

for col in numeric_cols:
    # Bỏ qua các biến Binary (chỉ có 0 và 1)
    if X[col].nunique() <= 2:
        continue
        
    series = X[col].dropna()
    if len(series) == 0: continue
    
    # Tính các chỉ số
    p01 = np.percentile(series, 1)
    p25 = np.percentile(series, 25)
    median = np.percentile(series, 50)
    p75 = np.percentile(series, 75)
    p99 = np.percentile(series, 99)
    maximum = series.max()
    minimum = series.min()
    
    # Tính độ lệch (Skewness) giữa Max và P99
    # Nếu Max gấp nhiều lần P99 -> Khả năng cao là Outlier cực đoan
    ratio_max_p99 = maximum / p99 if p99 > 0 else 0
    
    outlier_stats.append({
        'Column': col,
        'Min': minimum,
        'P01': p01,
        'Median': median,
        'P99': p99,
        'Max': maximum,
        'Max/P99 Ratio': round(ratio_max_p99, 2)
    })

# 2. Tạo DataFrame báo cáo và sắp xếp theo độ lệch giảm dần
outlier_report = pd.DataFrame(outlier_stats)
outlier_report = outlier_report.sort_values(by='Max/P99 Ratio', ascending=False)

# 3. Hiển thị Top 15 biến có Outlier nặng nhất
print("=== BÁO CÁO PHÁT HIỆN OUTLIER (TOP 15) ===")
print(outlier_report.to_string(index=False))

=== BÁO CÁO PHÁT HIỆN OUTLIER (TOP 15) ===
                Column           Min           P01       Median          P99          Max  Max/P99 Ratio
     N_AVG_DEPOSIT_12M  5.129000e+03  1.067510e+05 1.622795e+06 3.214021e+07 6.877327e+08          21.40
              BASE_AUM  5.342900e+04  5.495779e+05 6.416078e+06 8.386845e+07 8.834219e+08          10.53
  AMT_CASH_ADVANCE_12M  0.000000e+00  0.000000e+00 0.000000e+00 7.053942e+08 4.399059e+09           6.24
     AVG_DAYS_PAST_DUE  0.000000e+00  0.000000e+00 0.000000e+00 1.900000e+01 1.170000e+02           6.16
   CBAL_SHORTTERM_LOAN  4.890000e+02  2.602907e+06 1.379224e+08 8.593129e+08 3.987872e+09           4.64
     CNT_DPD_30PLUS_6M  0.000000e+00  0.000000e+00 0.000000e+00 2.000000e+00 9.000000e+00           4.50
    CBAL_LONGTERM_LOAN  1.480000e+02  2.603787e+06 1.378922e+08 8.618660e+08 3.573867e+09           4.15
              CBAL_AVG  2.314456e+07  7.603128e+07 3.106165e+08 1.335149e+09 5.276239e+09           3.95
            

In [30]:
# 1. Hàm tính ngưỡng (Chỉ dùng dữ liệu Train)
def get_capping_rules(df, cols, percentile=0.995):
    rules = {}
    for col in cols:
        if col in df.columns:
            # Tính giá trị phân vị trên tập Train
            limit = df[col].quantile(percentile)
            rules[col] = limit
            print(f"   -> Rule {col:<25}: Cap @ {limit:,.0f}")
    return rules

# 2. Hàm áp dụng (Dùng cho Train, OOS, OOT)
def apply_capping(df, rules):
    df_clean = df.copy()
    for col, limit in rules.items():
        if col in df_clean.columns:
            # Chỉ thay thế giá trị > limit bằng limit (không đụng vào giá trị nhỏ hơn)
            df_clean[col] = df_clean[col].clip(upper=limit)
    return df_clean

# Danh sách biến cần xử lý
cols_to_cap = [
    'N_AVG_DEPOSIT_12M', 'BASE_AUM', 'AMT_CASH_ADVANCE_12M',
    'AVG_DAYS_PAST_DUE', 'CBAL_SHORTTERM_LOAN', 'CNT_DPD_30PLUS_6M',
    'CBAL_LONGTERM_LOAN', 'CBAL_AVG', 'AFLIMT_MAX'
]

print(">>> [1] Đang học ngưỡng Outlier từ tập TRAIN (X)...")
capping_rules = get_capping_rules(X, cols_to_cap, percentile=0.995)

print("\n>>> [2] Đang áp dụng ngưỡng vào Train, OOS, OOT...")
X_capped = apply_capping(X, capping_rules)
X_oos_capped = apply_capping(X_oos, capping_rules)
X_oot_capped = apply_capping(X_oot, capping_rules)

# Kiểm tra nhanh kết quả trên OOT
print(f"\nKiểm tra biến {cols_to_cap[0]} trên OOT:")
print(f"   - Max trước khi cap: {X_oot[cols_to_cap[0]].max():,.0f}")
print(f"   - Max sau khi cap:   {X_oot_capped[cols_to_cap[0]].max():,.0f} (Phải bằng ngưỡng Train)")

>>> [1] Đang học ngưỡng Outlier từ tập TRAIN (X)...
   -> Rule N_AVG_DEPOSIT_12M        : Cap @ 44,081,819
   -> Rule BASE_AUM                 : Cap @ 110,680,615
   -> Rule AMT_CASH_ADVANCE_12M     : Cap @ 880,361,297
   -> Rule AVG_DAYS_PAST_DUE        : Cap @ 24
   -> Rule CBAL_SHORTTERM_LOAN      : Cap @ 1,032,608,513
   -> Rule CNT_DPD_30PLUS_6M        : Cap @ 3
   -> Rule CBAL_LONGTERM_LOAN       : Cap @ 1,035,228,584
   -> Rule CBAL_AVG                 : Cap @ 1,572,500,396
   -> Rule AFLIMT_MAX               : Cap @ 2,349,415,899

>>> [2] Đang áp dụng ngưỡng vào Train, OOS, OOT...

Kiểm tra biến N_AVG_DEPOSIT_12M trên OOT:
   - Max trước khi cap: 692,326,582
   - Max sau khi cap:   44,081,819 (Phải bằng ngưỡng Train)


In [31]:
X_capped.shape

(1137807, 37)

In [32]:
X_oos_capped.shape

(300317, 37)

In [33]:
X_oot_capped.shape

(302113, 37)

In [34]:
y_train = df['BAD_NEXT_12M']
y_oos = oos['BAD_NEXT_12M']
y_oot = oot['BAD_NEXT_12M']

X_capped.to_parquet('X_train.parquet', index=False)
X_oos_capped.to_parquet('X_oos.parquet', index=False)
X_oot_capped.to_parquet('X_oot.parquet', index=False)
y_train.to_frame().to_parquet('y_train.parquet', index=False)
y_oos.to_frame().to_parquet('y_oos.parquet', index=False)
y_oot.to_frame().to_parquet('y_oot.parquet', index=False)